In [1]:
import glob
import os
import sys
import random
import time
import numpy as np
import cv2
import math
from collections import deque
from keras.applications.xception import Xception
from keras.layers import Dense, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.models import Model
from keras.callbacks import TensorBoard

import tensorflow as tf
import keras.backend.tensorflow_backend as backend
from threading import Thread

from tqdm import tqdm
from PIL import Image,ImageFilter

from test_model import testModel


try:
    sys.path.append(glob.glob('E:\\CARLA_0.9.14\\WindowsNoEditor\\PythonAPI\\carla\\dist\\carla-0.9.14-py3.7-win-amd64.egg')[0])
except IndexError:
    pass

import carla



FPS=10
SHOW_PREVIEW = False
IM_WIDTH = 1024
IM_HEIGHT = 720
training_width = 256
training_height = 128
SECONDS_PER_EPISODE = 10
REPLAY_MEMORY_SIZE = 5_000
MIN_REPLAY_MEMORY_SIZE = 16
MINIBATCH_SIZE = 16
PREDICTION_BATCH_SIZE = 1
TRAINING_BATCH_SIZE = MINIBATCH_SIZE // 4
UPDATE_TARGET_EVERY = 5
MODEL_NAME = "Xception"

MEMORY_FRACTION = 0.4
MIN_REWARD = -100

EPISODES = 150

DISCOUNT = 0.9
epsilon = 0.001
EPSILON_DECAY = 0.9975 ## 0.9975 99975
MIN_EPSILON = 0.001

AGGREGATE_STATS_EVERY = 10

state_size = (128,256,3)


action_size =3
actions_steer=[0.15,-0.15,0]
actions_speed=[0.7]


SHOW = False

# Own Tensorboard class
class ModifiedTensorBoard(TensorBoard):

    # Overriding init to set initial step and writer (we want one log file for all .fit() calls)
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.step = 1
        self.writer = tf.summary.FileWriter(self.log_dir)

    # Overriding this method to stop creating default log writer
    def set_model(self, model):
        pass

    # Overrided, saves logs with our step number
    # (otherwise every .fit() will start writing from 0th step)
    def on_epoch_end(self, epoch, logs=None):
        self.update_stats(**logs)

    # Overrided
    # We train for one batch only, no need to save anything at epoch end
    def on_batch_end(self, batch, logs=None):
        pass

    # Overrided, so won't close writer
    def on_train_end(self, _):
        pass

    # Custom method for saving own metrics
    # Creates writer, writes custom metrics and closes writer
    def update_stats(self, **stats):
        self._write_logs(stats, self.step)
        self.step+=1


class CarEnv:
    SHOW_CAM = SHOW_PREVIEW
    STEER_AMT = 1.0
    im_width = IM_WIDTH
    im_height = IM_HEIGHT
    front_camera = None
    action = None
    
    fixed_delta_seconds = 0.1

    def __init__(self):
        self.client = carla.Client("localhost", 2000)
        self.client.set_timeout(10.0)
        self.world = self.client.get_world()

        traffic_manager = self.client.get_trafficmanager(8000)
        traffic_manager.set_synchronous_mode(True)

        settings = self.world.get_settings()
        settings.synchronous_mode = True
        settings.fixed_delta_seconds = self.fixed_delta_seconds
        settings.no_rendering_mode = True
        self.world.apply_settings(settings)

        self.blueprint_library = self.world.get_blueprint_library()
        self.model_3 = self.blueprint_library.filter("vehicle.carlamotors.firetruck")[0]

    def reset(self):
        self.list_lane = []
        self.actor_list = []
        self.collision_event=0

        self.transform = self.world.get_map().get_spawn_points()[random.randint(0,79)]
        self.vehicle = self.world.spawn_actor(self.model_3, self.transform)
        self.actor_list.append(self.vehicle)

        self.rgb_cam = self.blueprint_library.find('sensor.camera.rgb')
        self.rgb_cam.set_attribute("image_size_x", f"{self.im_width}")
        self.rgb_cam.set_attribute("image_size_y", f"{self.im_height}")
        self.rgb_cam.set_attribute("fov", f"90")

        transform = carla.Transform(carla.Location(x=4,z=1.6))

        self.sensor = self.world.spawn_actor(self.rgb_cam, transform, attach_to=self.vehicle)
        self.actor_list.append(self.sensor)
        self.sensor.listen(lambda data: self.process_img(data))

        self.vehicle.apply_control(carla.VehicleControl(throttle=0.0, brake=0.0))
        #time.sleep(4)

        #lanesensor = self.blueprint_library.find("sensor.other.lane_invasion")
        #self.Lane_sensor = self.world.spawn_actor(lanesensor, transform, attach_to=self.vehicle)
        #self.actor_list.append(self.Lane_sensor)
        #self.Lane_sensor.listen(lambda event: self.lane_data(event.crossed_lane_markings))
        
        colsensor = self.blueprint_library.find("sensor.other.collision")
        self.collision = self.world.spawn_actor(colsensor, transform, attach_to=self.vehicle)
        self.actor_list.append(self.collision)
        self.collision.listen(lambda event: self.collisionFunc(event))

        while self.front_camera is None:
            self.world.tick()
            time.sleep(0.01)
        
       

        #self.episode_start = time.time()
        self.vehicle.apply_control(carla.VehicleControl(throttle=0.0, brake=0.0))
        #self.world.tick()
        #time.sleep(self.fixed_delta_seconds)
    
        return self.front_camera

    def lane_data(self,lanes):
        for marking in lanes:
            self.list_lane.append(marking.type)

    def process_img(self, image):
        i = np.array(image.raw_data)
        #print(i.shape)
        i2 = i.reshape((self.im_height, self.im_width, 4))
        i3 = i2[:, :, :3]
        if self.SHOW_CAM:
            cv2.imshow("", i3)
            cv2.waitKey(1)


        #img=Image.fromarray(i3)
        #res=img.filter(ImageFilter.CONTOUR())

        self.front_camera = np.array(i3)
        
       
    
    def collisionFunc(self,e):
        self.collision_event=1

    
    def step(self,action_steer,action_speed):
        done=False
        reward=0
        
        #self.world.tick()
        #time.sleep(self.fixed_delta_seconds)
        #vehicle_location = self.vehicle.get_location()
        
    

        # if action_speed==0:
        #     reward=20
        # else:
        #     reward=-20
        
        
        self.vehicle.apply_control(carla.VehicleControl(throttle=actions_speed[action_speed],brake=0,steer=actions_steer[action_steer],reverse=0))
#         action = round(self.vehicle.get_control().steer,1)
#         action = actions_steer.index(action)
#         s = time.time()
#         while time.time() - s < 0.1:
        self.world.tick()
        #time.sleep(self.fixed_delta_seconds)
        #print("input: ",actions_steer[action_steer])
        
#         print("out: ",action)
#         min_d = 2
#         index_action = None
#         cnt=0
#         for ac in actions_steer:
#             if abs(action - ac) < min_d:
#                 min_d = action - ac
#                 index_action = cnt
#             cnt+=1
#         action = index_action
        #print("out: ",action)
        
        
        
#         self.world.tick()
#         time.sleep(self.fixed_delta_seconds)
        
        vehicle_location = self.vehicle.get_location()
        waypoint_road = self.world.get_map().get_waypoint(vehicle_location, project_to_road=True, 
                    lane_type=carla.LaneType.Driving)
        
        x_wp = waypoint_road.transform.location.x
        y_wp = waypoint_road.transform.location.y

        x_vh = vehicle_location.x
        y_vh = vehicle_location.y

        wp_array = np.array([x_wp, y_wp])
        vh_array = np.array([x_vh, y_vh])

        dist = np.linalg.norm(wp_array - vh_array)
        #print("distance= ",dist)

        waypoint_vehicle_yaw=self.vehicle.get_transform().rotation.yaw
        waypoint_vehicle_yaw_rad = math.radians(waypoint_vehicle_yaw)
        waypoint_vehicle_yaw_rad = waypoint_vehicle_yaw_rad % (2 * math.pi)

        waypoint_road_yaw = waypoint_road.transform.rotation.yaw
        waypoint_road_yaw_rad = math.radians(waypoint_road_yaw)
        waypoint_road_yaw_rad = waypoint_road_yaw_rad % (2 * math.pi)

        diff_yaw_rad = waypoint_vehicle_yaw_rad - waypoint_road_yaw_rad
        #diff_yaw_deg = diff_yaw_rad*180/math.pi
        #print("yaw= ",diff_yaw_deg)

        cos_yaw_diff=np.cos(diff_yaw_rad)
        reward = (1 * cos_yaw_diff) - (1 * dist) - (5*int(self.collision_event))
        
        #print(reward)
        
        if self.collision_event:
            done=True
            self.collision_event=0
        #print("reward= ",reward)
        
        #time.sleep(1.5)

        #print(self.list_lane)

#         for marking in self.list_lane:
#             if marking==carla.libcarla.LaneMarkingType.Broken or marking==carla.libcarla.LaneMarkingType.BrokenBroken :
#                 self.done=True
#                 done=True
#                 break
#                 reward=-10

#             elif marking==carla.libcarla.LaneMarkingType.Solid or marking==carla.libcarla.LaneMarkingType.Curb or marking==carla.libcarla.LaneMarkingType.Grass \
#                 or marking==carla.libcarla.LaneMarkingType.SolidSolid or marking==carla.libcarla.LaneMarkingType.SolidBroken or marking==carla.libcarla.LaneMarkingType.BrokenSolid:
#                 self.done=True
#                 done=True
#                 break

#         for _ in range(self.list_lane.count(carla.libcarla.LaneMarkingType.Broken)):
#             self.list_lane.remove(carla.libcarla.LaneMarkingType.Broken)
        
#         for _ in range(self.list_lane.count(carla.libcarla.LaneMarkingType.BrokenBroken)):
#             self.list_lane.remove(carla.libcarla.LaneMarkingType.BrokenBroken)
        
#         for _ in range(self.list_lane.count(carla.libcarla.LaneMarkingType.Solid)):
#             self.list_lane.remove(carla.libcarla.LaneMarkingType.Solid)
        
#         for _ in range(self.list_lane.count(carla.libcarla.LaneMarkingType.Curb)):
#             self.list_lane.remove(carla.libcarla.LaneMarkingType.Curb)
        
#         for _ in range(self.list_lane.count(carla.libcarla.LaneMarkingType.Grass)):
#             self.list_lane.remove(carla.libcarla.LaneMarkingType.Grass)

#         for _ in range(self.list_lane.count(carla.libcarla.LaneMarkingType.SolidSolid)):
#             self.list_lane.remove(carla.libcarla.LaneMarkingType.SolidSolid)
        
#         for _ in range(self.list_lane.count(carla.libcarla.LaneMarkingType.SolidBroken)):
#             self.list_lane.remove(carla.libcarla.LaneMarkingType.SolidBroken)
        
#         for _ in range(self.list_lane.count(carla.libcarla.LaneMarkingType.BrokenSolid)):
#             self.list_lane.remove(carla.libcarla.LaneMarkingType.BrokenSolid)
        
#        self.list_lane=[]
    
    
    

        
        return self.front_camera, reward, done


class DQNAgent:
    def __init__(self,state_size,action_size):
        self.state_size = state_size
        self.action_size = action_size
        
        self.model = self.create_model()
        self.target_model = self.create_model()
        self.target_model.set_weights(self.model.get_weights())

        self.replay_memory = deque(maxlen=REPLAY_MEMORY_SIZE)

        self.tensorboard = ModifiedTensorBoard(log_dir=f"logs/{MODEL_NAME}-{int(time.time())}")
        self.target_update_counter = 0
        self.graph = tf.compat.v1.get_default_graph()

        self.terminate = False
        self.last_logged_episode = 0
        self.training_initialized = False
        
        
        #self.reget()

    def create_model(self):
        base_model = Xception(weights=None, include_top=False, input_shape=(training_height, training_width,3))

        x = base_model.output
        x = GlobalAveragePooling2D()(x)

        predictions = Dense(self.action_size, activation="linear")(x)
        model = Model(inputs=base_model.input, outputs=predictions)
        model.compile(loss="mse", optimizer=Adam(lr=0.001), metrics=["accuracy"])
        return model

    def update_replay_memory(self, transition):
        # transition = (current_state, action, reward, new_state, done)
        self.replay_memory.append(transition)
        

    def train(self):
        if len(self.replay_memory) < MIN_REPLAY_MEMORY_SIZE:
            #print("Return")
            return
        #print(1)

        minibatch = random.sample(self.replay_memory, MINIBATCH_SIZE)

        current_states = np.array([transition[0] for transition in minibatch])/255
        with self.graph.as_default():
            current_qs_list = self.model.predict(current_states, PREDICTION_BATCH_SIZE)

        new_current_states = np.array([transition[3] for transition in minibatch])/255
        with self.graph.as_default():
            future_qs_list = self.target_model.predict(new_current_states, PREDICTION_BATCH_SIZE)

        X = []
        y = []

        for index, (current_state, action, reward, new_state, done) in enumerate(minibatch):
            if not done:
                max_future_q = np.max(future_qs_list[index])
                new_q = reward + DISCOUNT * max_future_q
            else:
                new_q = reward

            current_qs = current_qs_list[index]
            current_qs[action] = new_q

            X.append(current_state)
            y.append(current_qs)

        log_this_step = False
        if self.tensorboard.step > self.last_logged_episode:
            log_this_step = True
            self.last_log_episode = self.tensorboard.step

        with self.graph.as_default():
            self.model.fit(np.array(X)/255, np.array(y), batch_size=TRAINING_BATCH_SIZE, verbose=1, shuffle=False, callbacks=[self.tensorboard] if log_this_step else None)
            #print("fitting")

        if log_this_step:
            self.target_update_counter += 1

        if self.target_update_counter > UPDATE_TARGET_EVERY:
            self.target_model.set_weights(self.model.get_weights())
            self.target_update_counter = 0

    def get_qs(self, state):
        norm_state=np.array(state.reshape(-1,*state.shape))/255
        return self.model.predict(norm_state)[0]

    def train_in_loop(self):
        X = np.random.uniform(size=(1, training_height, training_width, 3)).astype(np.float32)
        y = np.random.uniform(size=(1, self.action_size)).astype(np.float32)
        with self.graph.as_default():
            self.model.fit(X,y, verbose=1, batch_size=1)

        self.training_initialized = True

        while True:
            if self.terminate:
                return
            self.train()
            time.sleep(0.01)
    def reget(self):
        self.model.load_weights(r"E:\study\graduation-proj\Carla-Imitation\ImitationFinal\PretrainStep #Done\PretrainedModelMaskServer10Epochs.ckpt")
        self.target_model.set_weights(self.model.get_weights())




if __name__ == '__main__':

    FPS = 10
    # For stats
    ep_rewards = [-1]

    # For more repetitive results
    random.seed(1)
    np.random.seed(1)
    tf.compat.v1.set_random_seed(1)

    # Memory fraction, used mostly when trai8ning multiple agents
    #gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=MEMORY_FRACTION)
    #backend.set_session(tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)))

    # Create models folder
    if not os.path.isdir('models'):
        os.makedirs('models')

    # Create agent and environment
    agent = DQNAgent(state_size,action_size)
    env = CarEnv()
    
    if SHOW:
        if not os.path.exists("Carla-Data-Test"):
            os.mkdir("Carla-Data-Test")


    # Start training thread and wait for training to be initialized
    trainer_thread = Thread(target=agent.train_in_loop, daemon=True)
    trainer_thread.start()
    while not agent.training_initialized:
        time.sleep(0.01)

    # Initialize predictions - forst prediction takes longer as of initialization that has to be done
    # It's better to do a first prediction then before we start iterating over episode steps
    #agent.get_qs(np.ones((env.im_height, env.im_width, 3)))
    
    num=1
    # Iterate over episodes
    for episode in tqdm(range(1, EPISODES + 1), ascii=True, unit='episodes'):
        #try:
            finish = False
            env.list_lane = []

            # Update tensorboard step every episode
            agent.tensorboard.step = episode

            # Restarting episode - reset episode reward and step number
            episode_reward = 0
            step = 1
            
            
            env.front_camera=None
        
            # Reset environment and get initial state
            current_state = env.reset()
            current_state=testModel(current_state)
            current_state = np.array(Image.fromarray(current_state).resize((training_width,training_height)))
            
            #cv2.imwrite(f'D:\\jupyter\\lanenet-lane-detection-pytorch-main\\mostafa\\1.png', current_state)

            #current_state = np.expand_dims(current_state.reshape(256, 512, 1), axis=0)

            # Reset flag and start iterating until episode ends
            done = False
            #episode_start = time.time()
            

            # Play for given number of seconds only
            while True:
                
                num+=1
                if SHOW:
                    str_num=str(num)
                    while len(str_num)<5:
                        str_num="0"+str_num

                    os.mkdir("Carla-Data-Test\\"+str_num)
                    cv2.imwrite("Carla-Data-Test\\"+str_num+"\\current.png", current_state)

                # This part stays mostly the same, the change is to query a model for Q values
                rand = np.random.random()
                flag=0
                if rand > epsilon:
                    # Get action from Q table
                    action = np.argmax(agent.get_qs(current_state))
                    flag=1
                else:
                    # Get random action
                    action = np.random.randint(0, action_size)
                    
                    # This takes no time, so we add a delay matching 60 FPS (prediction above takes longer)
                    

                #vehicle_location = env.vehicle.get_location()
                

                new_state, reward, done = env.step(action,0) #steer,speed
#                 cv2.imshow("current",current_state)
#                 cv2.waitKey(0)
#                 cv2.destroyAllWindows()
#                 print(actions_steer[action%7])
                
                
                
                
                new_state = testModel(new_state)
                new_state = np.array(Image.fromarray(new_state).resize((training_width,training_height)))
                
#                 cv2.imshow("next",new_state)
#                 cv2.waitKey(0)
#                 cv2.destroyAllWindows()
                if SHOW:
                    cv2.imwrite("Carla-Data-Test\\"+str_num+"\\next.png", new_state)
                    with open("Carla-Data-Test\\"+str_num+"\\info.txt",'w') as f1:
                        f1.write(str(reward)+'\n')
                        f1.write(str(done)+'\n')
                        f1.write(str(actions_steer[action])+'\n') #steer
                        f1.write(str(actions_speed[0])+'\n') #speed
                        if flag==0:
                            f1.write("random")
                        else:
                            f1.write("exact")
                
                #cv2.imwrite(f'D:\\jupyter\\lanenet-lane-detection-pytorch-main\\mostafa\\{num}.png', new_state)

                #new_state = np.expand_dims(new_state.reshape(256, 512, 1), axis=0)


                # Transform new continous state to new discrete state and count reward
                episode_reward += reward

                # Every step we update replay memory
                agent.update_replay_memory((current_state, action, reward, new_state, done))

                current_state = new_state
                step += 1

                if done:
                    break
                
            # End of episode - destroy agents
            for actor in env.actor_list:
                actor.destroy()

            # Append episode reward to a list and log stats (every given number of episodes)
            ep_rewards.append(episode_reward)
            if not episode % AGGREGATE_STATS_EVERY or episode == 1:
                average_reward = sum(ep_rewards[-AGGREGATE_STATS_EVERY:])/len(ep_rewards[-AGGREGATE_STATS_EVERY:])
                min_reward = min(ep_rewards[-AGGREGATE_STATS_EVERY:])
                max_reward = max(ep_rewards[-AGGREGATE_STATS_EVERY:])
                agent.tensorboard.update_stats(reward_avg=average_reward, reward_min=min_reward, reward_max=max_reward, epsilon=epsilon)

                # Save model, but only when min reward is greater or equal a set value
#                 if min_reward >= MIN_REWARD:
#                     agent.model.save_weights(f"models\\{num}.h5")
#                     pass

            # Decay epsilon
            if epsilon > MIN_EPSILON:
                epsilon *= EPSILON_DECAY
                epsilon = max(MIN_EPSILON, epsilon)
                print(epsilon)
            
    # Set termination flag for training thread and wait for it to finish
    agent.terminate = True
    trainer_thread.join()
    agent.model.save_weights("final.ckpt")


Using TensorFlow backend.



  0%|                                                                                    | 0/150 [00:00<?, ?episodes/s]

  2%|#3                                                                   | 3/150 [1:34:47<77:24:24, 1895.68s/episodes]


In [2]:
from collections import deque
a=deque(maxlen=2)
a.append(1)
len(a)

1

In [3]:
import pickle 


# Open a file and use dump() 
with open('ReplayMemoryData.pkl', 'wb') as file: 
    pickle.dump(agent.replay_memory, file) 